In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.train.Saver class: provides methods to save and restore models. 

tf.saved_model.simple_save function: easy way to build a saved model suitable for serving. 

Estimators automatically save and restore variables in the model_dir.

The tf.train.Saver constructor adds save and restore ops to the graph for all, or a specified list, of the variables in the graph.

TensorFlow saves variables in binary checkpoint files that map variable names to tensor values.

Create a Saver with tf.train.Saver() to manage all variables in the model. For example, the following snippet demonstrates how to call the tf.train.Saver.save method to save variables to checkpoint files:

In [2]:
# Create some variables.
v1 = tf.get_variable("v1", shape=[3], initializer = tf.zeros_initializer)
v2 = tf.get_variable("v2", shape=[5], initializer = tf.zeros_initializer)

inc_v1 = v1.assign(v1+1) #requires function to change value since a tf variable
dec_v2 = v2.assign(v2-1)

# Add an op to initialize the variables.
init_op = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, initialize the variables, do some work, and save the
# variables to disk.
with tf.Session() as sess:
  sess.run(init_op)
  # Do some work with the model.
  inc_v1.op.run()     # interesting
  dec_v2.op.run()
  # Save the variables to disk.
  save_path = saver.save(sess, "./models/model.ckpt")
  print("Model saved in path: %s" % save_path)

Model saved in path: ./models/model.ckpt


In [3]:
tf.reset_default_graph()

# Create some variables.
v1 = tf.get_variable("v1", shape=[3])
v2 = tf.get_variable("v2", shape=[5])

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
  # Restore variables from disk.
  saver.restore(sess, "./models/model.ckpt")
  print("Model restored.")
  # Check the values of the variables
  print("v1 : %s" % v1.eval())
  print("v2 : %s" % v2.eval())
    
    
# Above you can see how a tf.train.Saver object saved and restored a graph in its local directory

INFO:tensorflow:Restoring parameters from ./models/model.ckpt
Model restored.
v1 : [1. 1. 1.]
v2 : [-1. -1. -1. -1. -1.]


You can easily specify the names and variables to save or load by passing to the tf.train.Saver() constructor either of the following:

    A list of variables (which will be stored under their own names).
    A Python dictionary in which keys are the names to use and the values are the variables to manage.

In [4]:
tf.reset_default_graph()
# Create some variables.
v1 = tf.get_variable("v1", [3], initializer = tf.zeros_initializer)
v2 = tf.get_variable("v2", [5], initializer = tf.zeros_initializer)

# Add ops to save and restore only `v2` using the name "v2"
saver = tf.train.Saver({"v2": v2})   # Explicit saving

# Use the saver object normally after that.
with tf.Session() as sess:
  # Initialize v1 since the saver will not.
  v1.initializer.run()
  saver.restore(sess, "./models/model.ckpt")

  print("v1 : %s" % v1.eval())
  print("v2 : %s" % v2.eval())

INFO:tensorflow:Restoring parameters from ./models/model.ckpt
v1 : [0. 0. 0.]
v2 : [-1. -1. -1. -1. -1.]


In [5]:
# # We can quickly inspect variables in a checkpoint with the inspect_checkpoint library.

# # import the inspect_checkpoint library
# from tensorflow.python.tools import inspect_checkpoint as chkp

# # print all tensors in checkpoint file
# chkp.print_tensors_in_checkpoint_file("./models/model.ckpt", tensor_name='', all_tensors=True)

# # tensor_name:  v1
# # [ 1.  1.  1.]
# # tensor_name:  v2
# # [-1. -1. -1. -1. -1.]

# # print only tensor v1 in checkpoint file
# chkp.print_tensors_in_checkpoint_file("./models/model.ckpt", tensor_name='v1', all_tensors=False)

# # tensor_name:  v1
# # [ 1.  1.  1.]

# # print only tensor v2 in checkpoint file
# chkp.print_tensors_in_checkpoint_file("./models/model.ckpt", tensor_name='v2', all_tensors=False)

# # tensor_name:  v2
# # [-1. -1. -1. -1. -1.]


# # Not working but i'' believe it
# # All that is happening is loading in variables from a checkpoint, or specific ones

The easiest way to create a SavedModel is to use the tf.saved_model.simple_save function:

In [7]:
# simple_save(session,
#             export_dir,
#             inputs={"x": x, "y": y},    TEMPLATE
#             outputs={"z": z})

If your use case isn't covered by tf.saved_model.simple_save, use the manual builder APIs to create a SavedModel.

The tf.saved_model.builder.SavedModelBuilder class provides functionality to save multiple MetaGraphDefs. A MetaGraph is a dataflow graph, plus its associated variables, assets, and signatures. A MetaGraphDef is the protocol buffer representation of a MetaGraph. A signature is the set of inputs to and outputs from a graph.

In [8]:
# export_dir = ...
# ...
# builder = tf.saved_model.builder.SavedModelBuilder(export_dir)
# with tf.Session(graph=tf.Graph()) as sess:
#   ...
#   builder.add_meta_graph_and_variables(sess,
#                                        [tag_constants.TRAINING],
#                                        signature_def_map=foo_signatures,    TEMPLATE
#                                        assets_collection=foo_assets, 
#                                        strip_default_attrs=True)
# ...
# # Add a second MetaGraphDef for inference.
# with tf.Session(graph=tf.Graph()) as sess:
#   ...
#   builder.add_meta_graph([tag_constants.SERVING], strip_default_attrs=True)
# ...
# builder.save()

Can load in C++ and Python as well

Can use on Estimators too

And with CLI (command line interface)

https://www.tensorflow.org/programmers_guide/saved_model